In [ ]:
# Seattle Bikes — Modeling
Goal: build a simple baseline model on daily data with cross-validation.

Sections
1) Config & imports
2) Laai daily data
3) Feature engineering 
4) Bou X/y (train & test)
5) Choose CV (KFold vs TimeSeriesSplit) - gaan miskien skiep vir tyd
6) Fit & evaluate
7) Inspect coefficients (statsmodels)


In [ ]:
import pandas as pd
from pathlib import Path

daily_path = Path("../data/processed/daily.csv")
assert daily_path.exists(), f"Missing processed file: {daily_path}. Run 01_data_prep.ipynb first."

daily = pd.read_csv(daily_path, parse_dates=[0], index_col=0)
print("Loaded daily:", daily.shape, "| columns:", list(daily.columns))



In [ ]:
days = ['Mo','Tue','Wed', 'Thu', 'Fri', 'Sat', 'Sun' ]
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek==i).astype(float)

from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
start = daily.index.min().normalize()
end   = daily.index.max().normalize()
holidays = cal.holidays(start=start, end=end)

daily['holiday'] = daily.index.normalize().isin(holidays).astype(float)

In [ ]:
daily.head(3)

In [ ]:
import numpy as np
import pandas as pd

def hours_of_daylight_index(index, latitude=47.61, axis=23.44):
    """
    Vectorized daylight hours for a DatetimeIndex.
    Uses a simple axial-tilt model (approximate, not astronomical).
    """
    idx = pd.DatetimeIndex(index)
    # strip timezone if present (we only care about the date)
    if idx.tz is not None:
        idx = idx.tz_convert(None).tz_localize(None)
    # days since winter solstice (Dec 21, 2000)
    days = (idx - pd.Timestamp(2000, 12, 21)) / np.timedelta64(1, "D")
    phi = np.radians(latitude)
    decl = np.radians(axis) * np.cos(2 * np.pi * days / 365.25)
    m = 1.0 - np.tan(phi) * np.tan(decl)
    m = np.clip(m, 0.0, 2.0)
    daylight = 24.0 * np.degrees(np.arccos(1.0 - m)) / 180.0
    # clamp to [0, 24] just in case of rounding
    return np.clip(daylight, 0.0, 24.0).astype(float)

daily = daily.copy()
daily["daylight_hrs"] = hours_of_daylight_index(daily.index)
daily["daylight_hrs"].describe()

daily["daylight_hrs"].plot(title="Estimated daylight hours"); 


In [ ]:
daily['annual'] = (daily.index - daily.index[0]).days / 365.

daily.head(5)


In [ ]:
# ek is bewus dat om 'n weer (temperatuur, reenval, ens) dataset te join met die een kan die model baie verbeter, maar 
# besluit daarteen omrede ek eintlik net 'n gevoel vir Python en Github weer wil kry. 
# canonical feature list (adjust if you truly used 'Tues'/'Mo')
expected = ['Mo','Tue','Wed','Thu','Fri','Sat','Sun','holiday','daylight_hrs','annual']

present = [c for c in expected if c in daily.columns]
missing = [c for c in expected if c not in daily.columns]
if missing:
    print("Missing features (check your column names):", missing)
   

X = daily[present].astype(float)
y = daily['Total'].astype(float)
print("Features used:", present)
print("X shape:", X.shape, "| y shape:", y.shape)


In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd


daily = daily.copy()
# linear regression, good enough for now boytjie.
model = LinearRegression(fit_intercept=False)  # Omrede ons dummy variables gebruik, sal dit multicollinearity veroorsaak as ons nog 'n intercept ook insit
model.fit(X, y)

#
daily['predicted'] = pd.Series(model.predict(X), index=daily.index)

ax = daily[['Total', 'predicted']].plot(alpha=0.45, figsize=(10,4))
ax.set_title("In-sample fit: Total vs Predicted")

from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

r2 = r2_score(y, daily['predicted'])
try:
    
    rmse = mean_squared_error(y, daily['predicted'], squared=False)
except TypeError:
  
    rmse = np.sqrt(mean_squared_error(y, daily['predicted']))

print(f"R² (in-sample): {r2:.4f}  |  RMSE: {rmse:.2f}")



In [ ]:
resid = daily['Total'] - daily['predicted']
ax = resid.plot(figsize=(10,3), title="Residuals (in-sample)")
ax.axhline(0, linestyle='--')


In [ ]:
import numpy as np
import pandas as pd

assert hasattr(X, "columns"), "X must be a DataFrame so we can label coefficients"
assert len(model.coef_) == X.shape[1], "coef count must match number of feature columns"

params = pd.Series(model.coef_, index=X.columns).sort_values(ascending=False)


params_df = (params.to_frame("coef")
                   .assign(abs=lambda d: d["coef"].abs())
                   .sort_values("abs", ascending=False))
display(params_df)

